In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ansys.mapdl.core import launch_mapdl
mapdl=launch_mapdl()
E = 210e9
nu = 0.3
YS = 240e6
b = 0.0001
beta = np.pi/12
print(mapdl)

Product:             Ansys Mechanical Enterprise Academic Student
MAPDL Version:       23.1
ansys.mapdl Version: 0.64.0



In [2]:
def calculateK (press,x,shift,a1,a2,theta1,theta2,t,c):
    mapdl.clear()
    mapdl.prep7()
    m0 = mapdl.k("", 0,-0.1)
    m1 = mapdl.k("",0,0.1)
    m6 = mapdl.k("", 0.1, 0.1)
    m7 = mapdl.k("", 0.1, -0.1)
    main = mapdl.a(m0,m1,m6,m7)
    us1 = np.tan(theta1)
    us2 = np.tan(theta1+beta)# lower side of crack 1
    us3 = np.tan(theta1-beta)# upper side of crack 2

    d = a1*(np.sin(theta1)-np.cos(theta1)*us2)
    e = a1*(np.sin(theta1)-np.cos(theta1)*us3)

    x1 = (b+d)/(us1-us2)
    y1 = us1*x1-b 
    x3 = (b-e)/(us3-us1)
    y3 = us1*x3+b 
    
    # Crack geometry for Crack 1
    k0 = mapdl.k("",0,-b +shift)
    k1 = mapdl.k("",x1,y1+shift)
    k2 = mapdl.k("",a1*np.cos(theta1),a1*np.sin(theta1)+shift)
    k3 = mapdl.k("",x3,y3+shift)
    k4 = mapdl.k("",0,b+shift)
    crack1 = mapdl.a(k0,k1,k2,k3,k4)
    aout = mapdl.asba(main, crack1)

    ### Geometry for Crack 2

    ls1 = np.tan(theta2)
    ls2 = np.tan(theta2+beta)# lower side of crack 1
    ls3 = np.tan(theta2-beta)# upper side of crack 2

    d = a2*(np.sin(theta2)-np.cos(theta2)*ls2)
    e = a2*(np.sin(theta2)-np.cos(theta2)*ls3)

    x1 = (b+d)/(ls1-ls2)
    y1 = ls1*x1-b 
    x3 = (b-e)/(ls3-ls1)
    y3 = ls1*x3+b 


    k0 = mapdl.k("",0,-b -shift)
    k1 = mapdl.k("",x1,y1-shift)
    k2 = mapdl.k("",a2*np.cos(theta2),a2*np.sin(theta2)-shift)
    k3 = mapdl.k("",x3,y3-shift)
    k4 = mapdl.k("",0,b-shift)
    crack2 = mapdl.a(k0,k1,k2,k3,k4)
    aout = mapdl.asba(aout, crack2)
    mapdl.vext(aout, dz=t)
   
    mapdl.et(1,"SOLID285")
    # Define the Mesh
    mapdl.smrtsize(2)
    mapdl.aesize("ALL", 0.003)
    mapdl.vmesh(1)
    #mapdl.eplot(vtk=True, show_edges=True, show_axes=False, line_width=2, background="w")
    # Using the SI units

    mapdl.units("SI")  # SI - International system (m, kg, s, K).
    # Define a material (nominal steel in SI)
    mapdl.mp("EX", 1, E)  # Elastic moduli in Pa (kg/(m*s**2))
    mapdl.mp("DENS", 1, 7800)  # Density in kg/m3
    mapdl.mp("NUXY", 1, nu)  # Poisson's Ratio
    # Fix the lower side.
    mapdl.nsel("S", "LOC", "Y", -0.1)
    mapdl.d("ALL", "UY")
    mapdl.d("ALL", "UZ")
    mapdl.d("ALL", "UX")

    mapdl.nsel("S", "LOC", "Y", 0.1)
    mapdl.cp(5, "UX", "ALL")

    mapdl.asel("S",'LOC',"Y",0.1)
    mapdl.sfa('ALL', '', 'PRES', -press)
    mapdl.allsel(mute=True)
    mapdl.run("/SOLU")
    mapdl.antype("STATIC")
    mapdl.solve()
    mapdl.finish(mute=True)
    
    # List parameters to post-process
    if c==1:
        mapdl.post1()
        mapdl.path('P1',3,5)
        node1 = mapdl.queries.node(a1*np.cos(theta1),a1*np.sin(theta1)+shift,0)
        xs1 = 0.6*a1
        ys1 = us1*xs1-b+shift
        node2 = mapdl.queries.node(xs1,ys1,0)
        node3 = mapdl.queries.node(0,-b+shift,0)
        mapdl.ppath(1, node1)
        mapdl.ppath(2, node2)
        mapdl.ppath(3, node3)
        mapdl.local(11,0,a1*np.cos(theta1),a1*np.sin(theta1)+shift,0,theta1*180/np.pi)
        mapdl.csys(11)
        k1 = mapdl.kcalc(0,1,0,0)
        result = mapdl.result
        words = k1.split(' ')
        # Extracting the K1 value from the String output
        for i in range(len(words)):
            if words[i]=='KI':
                x=i
        K1 = words[x+3][:-1] 
        print(K1)
    elif c==2:
        mapdl.post1()
        mapdl.path('P2',3,5)
        node4 = mapdl.queries.node(a2*np.cos(theta2),a2*np.sin(theta2)-shift,0)
        xs2 = 0.6*a2
        ys2 = ls1*xs2+b-shift
        node5 = mapdl.queries.node(xs2,ys2,0)
        node6 = mapdl.queries.node(0,b-shift,0)
        mapdl.ppath(1, node4)
        mapdl.ppath(2, node5)
        mapdl.ppath(3, node6)
        mapdl.local(12,0,a2*np.cos(theta2),a2*np.sin(theta2)-shift,0,theta2*180/np.pi)
        mapdl.post1()
        mapdl.csys(12)
        k2 = mapdl.kcalc(0,1,0,0)

        result = mapdl.result
        words = k2.split(' ')
        # Extracting the K1 value from the String output
        for i in range(len(words)):
            if words[i]=='KI':
                x=i
        K1 = words[x+3][:-1] 
        print(K1)
   
    return K1


In [3]:
A = calculateK(250e6,0.005,0.01,0.02,0.02,np.pi/10,-np.pi/9,0.005,2)

0.67095E+08


In [6]:
P = [600e5,1000e5,1400e5]
A = [0.015,0.02,0.025]
S = [0.02,0.03,0.04]
TH = [-np.pi/10,np.pi/20,np.pi/10]
T = [0.01,0.015,0.02]

data = np.zeros((2187,10))
count =0
for i in range(len(P)):
    for k in range(len(S)):
        for l in range(len(A)):
            for m in range(len(A)):
                for n in range(len(TH)):
                    for o in range(len(TH)):
                        for p in range(len(T)):
                            print(count)
                            K= [0,0]
                            for q in range(len(K)):
                                K[q] = calculateK(P[i],0,S[k],A[l],A[m],TH[n],TH[o],T[p],q+1)   
                            data[count,0] = P[i]
                            data[count,1] = 0
                            data[count,2] = 2*S[k]
                            data[count,3] = A[l]
                            data[count,4] = A[m]
                            data[count,5] = TH[n]
                            data[count,6] = TH[o]
                            data[count,7] = T[p]
                            data[count,8] = K[0]
                            data[count,9] = K[1]
                            count=count+1        
      
            

0
0.99080E+07
0.13683E+08
1
0.10586E+08
0.14021E+08
2
0.11123E+08
0.13331E+08
3
0.89490E+07
0.12501E+08
4
0.10332E+08
0.12502E+08
5
0.92720E+07
0.12415E+08
6
0.95348E+07
0.11012E+08
7
0.99682E+07
0.11046E+08
8
0.95455E+07
0.10930E+08
9
0.13094E+08
0.13785E+08
10
0.13577E+08
0.13494E+08
11
0.12615E+08
0.12786E+08
12
0.12239E+08
0.13424E+08
13
0.11855E+08
0.13153E+08
14
0.12104E+08
0.12977E+08
15
0.12777E+08
0.11697E+08
16
0.12127E+08
0.11678E+08
17
0.12131E+08
0.11410E+08
18
0.12794E+08
0.14258E+08
19
0.13512E+08
0.13739E+08
20
0.12596E+08
0.13750E+08
21
0.12225E+08
0.13576E+08
22
0.12934E+08
0.14174E+08
23
0.12193E+08
0.13462E+08
24
0.12243E+08
0.11711E+08
25
0.12745E+08
0.11991E+08
26
0.12107E+08
0.11927E+08
27
0.88860E+07
0.41802E+08
28
0.87596E+07
0.40902E+08
29
0.97224E+07
0.41657E+08
30
0.63512E+07
0.17469E+08
31
0.68420E+07
0.17282E+08
32
0.71383E+07
0.17926E+08
33
0.64154E+07
0.30949E+08
34
0.67152E+07
0.29034E+08
35
0.65623E+07
0.30550E+08
36
0.11189E+08
0.42151E+08
37
0.11744E

0.32703E+08
297
0.97547E+07
0.24815E+08
298
0.86761E+07
0.24363E+08
299
0.95541E+07
0.24411E+08
300
0.72011E+07
0.24577E+08
301
0.73887E+07
0.23847E+08
302
0.71110E+07
0.24667E+08
303
0.73828E+07
0.21264E+08
304
0.76660E+07
0.21668E+08
305
0.73302E+07
0.21252E+08
306
0.11509E+08
0.24785E+08
307
0.10826E+08
0.24017E+08
308
0.10413E+08
0.24218E+08
309
0.10434E+08
0.24888E+08
310
0.94245E+07
0.23795E+08
311
0.98919E+07
0.25141E+08
312
0.10021E+08
0.21337E+08
313
0.95894E+07
0.21906E+08
314
0.96889E+07
0.21317E+08
315
0.11286E+08
0.24872E+08
316
0.11718E+08
0.24602E+08
317
0.11256E+08
0.24619E+08
318
0.93903E+07
0.25817E+08
319
0.10475E+08
0.25247E+08
320
0.10401E+08
0.24746E+08
321
0.10259E+08
0.22189E+08
322
0.11002E+08
0.22547E+08
323
0.10751E+08
0.22617E+08
324
0.35296E+08
0.13796E+08
325
0.34239E+08
0.14675E+08
326
0.36069E+08
0.14442E+08
327
0.34857E+08
0.13210E+08
328
0.35088E+08
0.13198E+08
329
0.34052E+08
0.13439E+08
330
0.34848E+08
0.11295E+08
331
0.34814E+08
0.12519E+08
332
0.33

0.15011E+08
590
0.45910E+08
0.14384E+08
591
0.45803E+08
0.13636E+08
592
0.45869E+08
0.13273E+08
593
0.46182E+08
0.13848E+08
594
0.35404E+08
0.43213E+08
595
0.35560E+08
0.43542E+08
596
0.36028E+08
0.42662E+08
597
0.35517E+08
0.18306E+08
598
0.35479E+08
0.18610E+08
599
0.34748E+08
0.18326E+08
600
0.35537E+08
0.32849E+08
601
0.35087E+08
0.32727E+08
602
0.35238E+08
0.32572E+08
603
0.17686E+08
0.42978E+08
604
0.16995E+08
0.43345E+08
605
0.15973E+08
0.43961E+08
606
0.16982E+08
0.18638E+08
607
0.16485E+08
0.18733E+08
608
0.15832E+08
0.19531E+08
609
0.16542E+08
0.33039E+08
610
0.17406E+08
0.33415E+08
611
0.15879E+08
0.32628E+08
612
0.46916E+08
0.43451E+08
613
0.46604E+08
0.43081E+08
614
0.47118E+08
0.43986E+08
615
0.46555E+08
0.18531E+08
616
0.46667E+08
0.19305E+08
617
0.46298E+08
0.18922E+08
618
0.45651E+08
0.33715E+08
619
0.46812E+08
0.33732E+08
620
0.45946E+08
0.32933E+08
621
0.36439E+08
0.25106E+08
622
0.36039E+08
0.24254E+08
623
0.36491E+08
0.24298E+08
624
0.36058E+08
0.23885E+08
625
0.36

0.40665E+08
883
0.72874E+08
0.40359E+08
884
0.72677E+08
0.40306E+08
885
0.66214E+08
0.37907E+08
886
0.65352E+08
0.36856E+08
887
0.66216E+08
0.40054E+08
888
0.64099E+08
0.33767E+08
889
0.63222E+08
0.32070E+08
890
0.64584E+08
0.34809E+08
891
0.30529E+08
0.17261E+08
892
0.29972E+08
0.16440E+08
893
0.31809E+08
0.16695E+08
894
0.29047E+08
0.13057E+08
895
0.29967E+08
0.13412E+08
896
0.31209E+08
0.13105E+08
897
0.30286E+08
0.97746E+07
898
0.30670E+08
0.93485E+07
899
0.30027E+08
0.10173E+08
900
0.35025E+08
0.21017E+08
901
0.34328E+08
0.19530E+08
902
0.35548E+08
0.20498E+08
903
0.35936E+08
0.16573E+08
904
0.34415E+08
0.18532E+08
905
0.35264E+08
0.17167E+08
906
0.37521E+08
0.14521E+08
907
0.34478E+08
0.14817E+08
908
0.35982E+08
0.14362E+08
909
0.35903E+08
0.20927E+08
910
0.34159E+08
0.20643E+08
911
0.33596E+08
0.20113E+08
912
0.35765E+08
0.19386E+08
913
0.33685E+08
0.18148E+08
914
0.34870E+08
0.19163E+08
915
0.35842E+08
0.16663E+08
916
0.34125E+08
0.15829E+08
917
0.34190E+08
0.16574E+08
918
0.27

0.26582E+08
0.47987E+08
1170
0.34422E+08
0.71255E+08
1171
0.33546E+08
0.70141E+08
1172
0.32251E+08
0.69431E+08
1173
0.32863E+08
0.29393E+08
1174
0.32223E+08
0.30114E+08
1175
0.35190E+08
0.29706E+08
1176
0.34899E+08
0.51822E+08
1177
0.33009E+08
0.52319E+08
1178
0.33673E+08
0.50595E+08
1179
0.34674E+08
0.71615E+08
1180
0.33066E+08
0.70764E+08
1181
0.33889E+08
0.71695E+08
1182
0.33980E+08
0.29967E+08
1183
0.31049E+08
0.29416E+08
1184
0.32012E+08
0.29749E+08
1185
0.34009E+08
0.52739E+08
1186
0.31191E+08
0.52966E+08
1187
0.32518E+08
0.52654E+08
1188
0.25844E+08
0.39277E+08
1189
0.27101E+08
0.40787E+08
1190
0.25693E+08
0.40075E+08
1191
0.24122E+08
0.35856E+08
1192
0.23047E+08
0.35698E+08
1193
0.24331E+08
0.36752E+08
1194
0.24325E+08
0.31210E+08
1195
0.23768E+08
0.32744E+08
1196
0.23998E+08
0.30743E+08
1197
0.32009E+08
0.39676E+08
1198
0.30151E+08
0.39525E+08
1199
0.31488E+08
0.40350E+08
1200
0.29235E+08
0.38890E+08
1201
0.30303E+08
0.37057E+08
1202
0.30553E+08
0.38328E+08
1203
0.29841E+08
0.

0.31627E+08
0.39787E+08
1453
0.31451E+08
0.41123E+08
1454
0.29120E+08
0.40713E+08
1455
0.31810E+08
0.35803E+08
1456
0.29160E+08
0.35672E+08
1457
0.29691E+08
0.35882E+08
1458
0.23119E+08
0.31927E+08
1459
0.24700E+08
0.32715E+08
1460
0.25955E+08
0.31105E+08
1461
0.20881E+08
0.29168E+08
1462
0.24108E+08
0.29172E+08
1463
0.21635E+08
0.28967E+08
1464
0.22248E+08
0.25695E+08
1465
0.23259E+08
0.25774E+08
1466
0.22273E+08
0.25504E+08
1467
0.30552E+08
0.32166E+08
1468
0.31680E+08
0.31487E+08
1469
0.29436E+08
0.29835E+08
1470
0.28558E+08
0.31322E+08
1471
0.27663E+08
0.30690E+08
1472
0.28243E+08
0.30281E+08
1473
0.29812E+08
0.27294E+08
1474
0.28296E+08
0.27248E+08
1475
0.28306E+08
0.26624E+08
1476
0.29853E+08
0.33269E+08
1477
0.31528E+08
0.32057E+08
1478
0.29391E+08
0.32084E+08
1479
0.28526E+08
0.31676E+08
1480
0.30179E+08
0.33073E+08
1481
0.28450E+08
0.31412E+08
1482
0.28566E+08
0.27325E+08
1483
0.29738E+08
0.27980E+08
1484
0.28250E+08
0.27830E+08
1485
0.20734E+08
0.97538E+08
1486
0.20439E+08
0.

0.21297E+08
0.74622E+08
1736
0.22397E+08
0.76022E+08
1737
0.29414E+08
0.10289E+09
1738
0.29127E+08
0.10363E+09
1739
0.27577E+08
0.10356E+09
1740
0.27935E+08
0.44114E+08
1741
0.26463E+08
0.46605E+08
1742
0.26427E+08
0.43148E+08
1743
0.26572E+08
0.75357E+08
1744
0.25699E+08
0.75438E+08
1745
0.26799E+08
0.77053E+08
1746
0.28962E+08
0.10225E+09
1747
0.29334E+08
0.10242E+09
1748
0.28037E+08
0.10251E+09
1749
0.28697E+08
0.44262E+08
1750
0.29444E+08
0.43896E+08
1751
0.27256E+08
0.44925E+08
1752
0.26749E+08
0.78102E+08
1753
0.28269E+08
0.76422E+08
1754
0.28557E+08
0.76306E+08
1755
0.22761E+08
0.57901E+08
1756
0.20244E+08
0.56848E+08
1757
0.22293E+08
0.56958E+08
1758
0.16803E+08
0.57346E+08
1759
0.17240E+08
0.55643E+08
1760
0.16592E+08
0.57557E+08
1761
0.17226E+08
0.49616E+08
1762
0.17887E+08
0.50558E+08
1763
0.17104E+08
0.49587E+08
1764
0.26855E+08
0.57831E+08
1765
0.25262E+08
0.56040E+08
1766
0.24298E+08
0.56509E+08
1767
0.24345E+08
0.58072E+08
1768
0.21991E+08
0.55521E+08
1769
0.23081E+08
0.

0.27278E+08
0.55720E+08
2019
0.26449E+08
0.59072E+08
2020
0.27315E+08
0.59928E+08
2021
0.27322E+08
0.57586E+08
2022
0.25679E+08
0.52811E+08
2023
0.25343E+08
0.53130E+08
2024
0.25488E+08
0.51354E+08
2025
0.81445E+08
0.34817E+08
2026
0.82829E+08
0.34240E+08
2027
0.83157E+08
0.34706E+08
2028
0.82476E+08
0.32592E+08
2029
0.81313E+08
0.34262E+08
2030
0.81933E+08
0.34735E+08
2031
0.82885E+08
0.29475E+08
2032
0.83479E+08
0.29606E+08
2033
0.80276E+08
0.29937E+08
2034
0.40740E+08
0.35637E+08
2035
0.42044E+08
0.34687E+08
2036
0.43058E+08
0.35019E+08
2037
0.42005E+08
0.33591E+08
2038
0.40415E+08
0.35678E+08
2039
0.41451E+08
0.34671E+08
2040
0.43273E+08
0.32303E+08
2041
0.41997E+08
0.30842E+08
2042
0.40334E+08
0.30281E+08
2043
0.10715E+09
0.33817E+08
2044
0.10708E+09
0.34134E+08
2045
0.10736E+09
0.34894E+08
2046
0.10696E+09
0.32739E+08
2047
0.10705E+09
0.35026E+08
2048
0.10712E+09
0.33562E+08
2049
0.10687E+09
0.31817E+08
2050
0.10703E+09
0.30970E+08
2051
0.10776E+09
0.32312E+08
2052
0.82610E+08
0.

In [7]:
# saving the data matrix as npy 
np.save('dataPhase02Updated.npy',data)

In [17]:
len(data)

5184